![](../images/rivacon_frontmark_combined_header.png)

# Inflation Linked Bonds
This notebook demonstrates the pricing of Inflation Linked Bonds using the pyvacon module. The notebook first starts with an introduction to the pricing of simple plain vanilla bonds as a basis of the Inflation Linked Bond pricing and to familiarize the reader with the basic concepts of pricing in the pyvacon module. In the second we then focus on the pricing of Inflation Linked bonds.

In [ ]:
# import necesary modules
import pyvacon
import datetime as dt

import matplotlib.pyplot as plt
%matplotlib inline

import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.models.plot as model_plot
import pyvacon.models.tools as model_tools

print('Imported pyvacon version: ' + pyvacon.version.version)

In [ ]:
refdate = dt.datetime(2017,2,20)
pyvacon.analytics.setLogLevel('DEBUG') # set loglevel
pyvacon.analytics.registerSerialization('') #register all classes for serialization

## Bonds
This section discusses the pricing of simple plain vanilla bonds as prerequisite to the Inflation Linked bond pricing. It introduces the basic concepts such as Specifications, Pricer, PricingParameter.

### Setup Specifications
The static data of all products that are modeled in pyvacon is defined in so-called Specifications. Each producttype is represented by a specialized Specification class that inherits from the BaseSpecification class. Simple plain vanilla bonds are captured by the *BondSpecification* and we show in the following how different simple types of bonds can be defined using this class.

#### Zero Bond

In [ ]:
expiry = refdate + dt.timedelta(days=5*365)
zero_bond = pyvacon.instruments.BondSpecification('Zero_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR', expiry, refdate, 100.0, 
                            enums.DayCounter.ACT365_FIXED, [], [], '', [], [])


Most of the pyvacon classes offer a method *get_dictionary* that lists the contents of the swigged C++ object as a dictionary. Note that the simple python magic function *__dict__* would not work since the python object is just a wrapped C++ object.

In [ ]:
zero_bond.get_dictionary()

#### Fixed Coupon Bond

In [ ]:
expiry = refdate + dt.timedelta(days=5*365)
coupon_dates = [dt.datetime(y, 2, 20) for y in [2018,2019,2020,2021]]
coupons = [0.05]*len(coupon_dates)
fixed_coupon_bond = pyvacon.instruments.BondSpecification('Fixed_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR', expiry, refdate, 100.0, 
                            enums.DayCounter.ACT365_FIXED, coupon_dates, coupons, '', [], [])

#### Float Coupon Bond

In [ ]:
coupon_dates = [dt.datetime(y, 2, 20) for y in [2018,2019,2020,2021]]
spreads = [1.0]*len(coupon_dates)
float_coupon_bond = pyvacon.instruments.BondSpecification('Float_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR', expiry, refdate, 100.0, 
                            enums.DayCounter.ACT365_FIXED, [], [], 'EUR6M', coupon_dates, 
                            spreads)

#### Fixed-To-Float

In [ ]:
float_coupon_dates = [dt.datetime(y, 2, 20) for y in [2022,2023,2024]]
float_spreads = [0.5]*3
fix_to_float_bond = pyvacon.instruments.BondSpecification('Fixed_to_Float_Coupon', 'DBK', enums.SecuritizationLevel.NONE, 'EUR', expiry, refdate, 100.0, 
                            enums.DayCounter.ACT365_FIXED, coupon_dates, coupons, 'EUR6M', float_coupon_dates, 
                            float_spreads)

### Pricing
To price a product, the *PricingData* object for this kind of product must be constructed. This object contains all data for pricing:
- The respective *Specification*
- *PricingParameter* that may contain numerical parameter as well as e.g. the model that shall be used
- The market data used to price the respective product
- The *PricingRequest* that defines what values shall be computed by the pricer, e.g. Price, Modified Duration etc.
- The valuation date

In [ ]:
pricing_data_simple = pyvacon.pricing.BondPricingData()
pricing_data_simple.param = pyvacon.pricing.BondPricingParameter()
pricing_data_simple.param.useJLT = False
pricing_data_simple.discountCurve = mkt_testdata.InterestRate.Curves.EONIA(refdate)
pricing_data_simple.pricingRequest = pyvacon.pricing.PricingRequest()
pricing_data_simple.pricingRequest.setCleanPrice(True)
pricing_data_simple.pricer = 'BondPricer'
pricing_data_simple.valDate = refdate

There are two different models to calculate the fair price of a bond. The simple discounted cashflow method and the Jarrow-Landau-Turnbull (JLT) model.
The  Discounted Cashflow method uses a (risk-adjusted) discount curve to discount all expected cashflows. To determine the expected cashflow of a floating coupon, the pricer just computes the forward rate from a explicitely defined fixing curve (also of type Discount curve). Let us assume that the bond has $N+1$ cashflows $c_i$, $1\leq i\leq N+1$ where the last cashflow is the payment of the principal. The cashflow $c_i$ occurs at time $t_i$. Then, the value of the bond is given by
$$
V=\sum_i df_i\cdot c_i
$$
where $df_i$ is the discount factor at timepoint $i$.

The JLT model represents the credit view on bond pricing. It assumes that there is a survival curve for the respective issuer and a recovery rate for the bond and then prices the bond by summing all cashflows weighted by their respective probability: The cashflows stemming from recovery in case of default weighted by default probabilities and the cashflows occurring if the issuer survives.

Both methods are implemented in pyvacon. Since we are interested in principles of Inflation Linked Bond pricing we restrict ourselves to the Discounted Cashflows models for the ease of presentation.
#### Discounted Cashflow

In [ ]:
# pricing_data_simple: pricing data for simple pv calculation using discount curve
#mkt_testdata.InterestRate.EUR12M()
pricing_data_simple.spec = zero_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))

In [ ]:
pricing_data_simple.spec = fixed_coupon_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))

In [ ]:
pricing_data_simple.spec = float_coupon_bond
pricing_data_simple.fixingCurve = mkt_testdata.InterestRate.Curves.EONIA(refdate)
pricing_data_simple.pastFixing = 0.1
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))

In [ ]:
pricing_data_simple.spec = fix_to_float_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))

#### Outputs
##### Macaulay Duration
Macaulay duration, named for Frederick Macaulay who introduced the concept, is the weighted average maturity of cash flows. Consider some set of fixed cash flows. The present value of these cash flows is:

$$ V=\sum _{i=1}^{n}PV_{i}V=\sum _{i=1}^{n}PV_{i}$$
The Macaulay duration is defined as:
$$ MacD={\frac {\sum _{i=1}^{n}{t_{i}PV_{i}}}{\sum _{i=1}^{n}{PV_{i}}}}={\frac {\sum _{i=1}^{n}{t_{i}PV_{i}}}{V}}=\sum _{i=1}^{n}t_{i}{\frac {PV_{i}}{V}}$$
where:

- $i$ indexes the cash flows,
- $PV_{i}$ is the present value of the $i$th cash payment from an asset,
- $t_{i}$ is the time in years until the $i$th payment will be received,
- $V$ is the present value of all future cash payments from the asset.

<div class="alert alert-block alert-warning">
<b>Note:</b> To compute the year fraction $t_i$ we use the daycounter of the underlying discount curve.
</div>

##### Modified Duration and Rho
In contrast to Macaulay duration, modified duration (sometimes abbreviated MD) is a price sensitivity measure, defined as the percentage derivative of price with respect to yield (the logarithmic derivative of bond price with respect to yield). Modified duration applies when a bond or other asset is considered as a function of yield. In this case one can measure the logarithmic derivative with respect to yield:

$$ ModD(y)\equiv -{\frac {1}{V}}\cdot {\frac {\partial V}{\partial y}}=-{\frac {\partial \ln(V)}{\partial y}}$$

When the yield is expressed continuously compounded, Macaulay duration and modified duration are numerically equal. To see this, if we take the derivative of price or present value, expression (2), with respect to the continuously compounded yield $y$ we see that:

$$\frac {\partial V}{\partial y}=-\sum _{i=1}^{n}t_{i}\cdot CF_{i}\cdot e^{-y\cdot t_{i}}=-MacD\cdot V,$$
In other words, for yields expressed continuously compounded,
$ModD=MacD$.

We apply the above equation with the difference that instead of the flat yield we use the time dependent zero rates $r(t_i)$ from the discount curve used to discount the cash flows:
$$\frac {\partial V}{\partial y}=-\sum _{i=1}^{n}t_{i}\cdot CF_{i}\cdot e^{-r(t_i)\cdot t_{i}}=-MacD\cdot V,$$.
To be consistent with the equity part we store this modified duration as rho in the PricingResults object.

In [ ]:
pricing_data_simple.pricingRequest.setMacaulayDuration(True)
pricing_data_simple.pricingRequest.setRho(True)
pricing_data_simple.spec = fix_to_float_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  Macaulay Duration: " + str(results.getMacaulayDuration()) + ",  Mod Duration/rho: " + str(results.getRho('EONIA')))

#### Convexity
Convexity is the second derivative of the price w.r.t. the rates, measuring.#

In [ ]:
pricing_data_simple.pricingRequest.setConvexity(True)
pricing_data_simple.spec = fix_to_float_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice())
      + ", convexity: " + str(results.getConvexity()))

##### Yield to maturity (YTM)
The yield to maturity (YTM), book yield or redemption yield of a bond is the (theoretical) internal rate of return (IRR, overall interest rate) earned by an investor who buys the bond today at the market price, assuming that the bond is held until maturity, and that all coupon and principal payments are made on schedule.

The YTM is often given in terms of Annual Percentage Rate (A.P.R.), but more often market convention is followed. In a number of major markets (such as gilts) the convention is to quote annualized yields with semi-annual compounding (see compound interest); thus, for example, an annual effective yield of 10.25% would be quoted as 10.00%, because 1.05 × 1.05 = 1.1025 and 2 × 5 = 10.

<div class="alert alert-block alert-warning">
<b>Note:</b> The YTM computed by the BondPricer is the continuously compounded rate w.r.t. the daycounter of the respective discount curve used for pricing the product.
</div>

In [ ]:
pricing_data_simple.pricingRequest.setYTM(True)
pricing_data_simple.spec = fixed_coupon_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  ytm: " + str(results.getYTM()))

##### Theta
Theta describes the change of value w.r.t. to time to maturity. 

In [ ]:
pricing_data_simple.pricingRequest.setYTM(False) #switch of ytm calculation
pricing_data_simple.pricingRequest.setTheta(True)
pricing_data_simple.spec = fix_to_float_bond
results = pyvacon.pricing.price(pricing_data_simple)
print(pricing_data_simple.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  theta: " + str(results.getTheta()))

## Inflation Linked Bonds
In the following we see how we may price an inflation linked bond. In contrast to the plain vanilla bonds described in the previous section, all payments of an inflation linked bond are adjusted  in relation to a Consumer Price Index (CPI) value or a Retail Prices Index (RPI) value for a country. 
Note that such an index is usually computed on a monthly basis from the government of the respective country. Since payments of coupons and notional of the bond are not necessarily on an EOM basis, an interpolation of these indices is necessary, which is determined in the respective bond's termsheet. We will discuss the different interpolation conventions in the next section.

### Interpolation of price indices
#### Linear Interpolation
One method often used is simple linear interpolation. Let $I(M_t)$ define the index value of the reference index for month $M_t$ belonging to time point $t$. Linear interpolation
for a day $d$ in the month $M_t$ is defined by 
$$
I(t) = I(M_t) +  \frac{t-1}{D}(I(M_t+1)-I(M_t))
$$
where $I(M_t+1)$ denotes the index belonging to the following month and $D$ is the number of days in the month $M_t$. This interpolation is most often used for bonds issued from Canada, USA, France, Germany, Italy and Sweden.

#### Japanese interpolation
We call the following formula Japanese interpolation because it is commonly used for Japanese bonds. Here, the interpolation is defined piecewise depending on the day of the month.
Let us denote the day of the month of timepoint $t$ as $d$. We have
$$
    I(t)=I(M_t) \mbox{for } d=10,
$$


$$
    I(t)=I(M_t)+ \frac{t-10}{D}\left(I(M_t+1)-I(M_t)\right) \mbox{for } d > 10,
$$
where here, **$D$** denotes the number of days between the 11th day of month $M_t$ and the 10th day of the following month,
$$
I(t)=I(M_t-1)+ \frac{\bar D}{D}(I(M_t)-I(M_t-1)) \mbox{for } d < 10,
$$
where $\bar D$ is the number of days from the 11th day of $M_t-1$ month to the 10th day of $M_t$ and $D$  denotes the number of days between the 11th day of month $M_t$ and the 10th day of the following month.


In [ ]:
# plot inflation curves
inflation_curve = mkt_testdata.Inflation.CPI(refdate)
dates = [refdate + dt.timedelta(days=i) for i in range(20, 2*30)]

german = [inflation_curve.value(refdate, 
                      dates[i], enums.InflationInterpolation.GERMAN) for i in range(len(dates))]

japan = [inflation_curve.value(refdate, 
                      dates[i], enums.InflationInterpolation.JAPAN) for i in range(len(dates))]
const = [inflation_curve.value(refdate, 
                      dates[i], enums.InflationInterpolation.CONSTANT) for i in range(len(dates))]
plt.plot(dates, const, '.', label = 'CONSTANT')
plt.plot(dates, german, '.', label = 'GERMAN')
plt.plot(dates, japan, '.', label = 'JAPAN')
plt.xticks(rotation=70)
legend = plt.legend()

In [ ]:
inflation_curve.value(refdate, refdate + dt.timedelta(days = 2*365), enums.InflationInterpolation.GERMAN)

### Pricing
#### Pricing Data

As explained in [Overview](../overview.ipynb), we need to setup the respective PricingData object which is *InflationLinkedBondPricingData* for inflation linkers. This object needs the following data
- **spec**: An InflationLinkedBondSpecification object
- **param**: BondPricingParameter that is mainly used to define the model, i.e. JLT or simple discounted cashflow
- **discountCurve**: The discount curve used to discount all cashflows (in case of JLT this is the risk free discount curve, otherwise the risk adjusted discount curve)
- **pricingRequest**: The pricing request defining which figures will be computed, e.g. YTM
- **inflationFwdCurve**: A InflationIndexForwardCurve containing the estimated forward monthly inflation indices
- **inflationFixingReference1**: The fixed inflation value needed to fix the base reference index by interpolation (only needed if base index ref date is before the valuation date)
- **inflationFixingReference2**: The second fixed inflation value for the base index (two values are needed due to interpolation)
- **inflationFixing1**: Fixed inflation index needed to compute inflation adjustment for the next cashflow
- **inflationFixing2**: Fixed inflation index needed to compute inflation adjustment for the next cashflow

In [ ]:
pricing_data_infl = pyvacon.pricing.InflationLinkedBondPricingData()
pricing_data_infl.param = pyvacon.pricing.BondPricingParameter()
pricing_data_infl.param.useJLT = False
pricing_data_infl.discountCurve = mkt_testdata.InterestRate.Curves.EONIA(refdate)
pricing_data_infl.pricingRequest = pyvacon.pricing.PricingRequest()
pricing_data_infl.pricingRequest.setPricingData(False)
pricing_data_infl.pricingRequest.setCleanPrice(True)
pricing_data_infl.inflationFwdCurve = inflation_curve
pricing_data_infl.inflationFixingReference1 = 100
pricing_data_infl.inflationFixingReference2 = 100
pricing_data_infl.inflationFixing1 = 10000
pricing_data_infl.inflationFixing2 = 10000
pricing_data_infl.valDate = refdate

#### InflationLinkedBondSpecification

The InflationLinkedBondSpecification is similar to the BondSpecification. It just adds some inflation related information
- **deflation protection**: This flag determines whether coupons and notional payments are also adjusted in case of deflation
- **inflation interpolation**: This specifies the interpolation method from the methods described [here](#Interpolation-of-price-indices) to derive the inflation index for a certain day in the month. The values that can be used here are defined in *enums.InflationInterpolation*.
- **inflation lag**: Needed to compute the month of the respective inflation index used to determine cashflow adjustment (it defines the number of month  before the cashflow month to derive inflation index month used).
- **inflation index**: Defines the inflation index used. This is only needed if automatic setup of pricing data will be used. Otherwise, just the inflation curve defined in the PricingData will be used and this string will be ignored.


In [ ]:
coupon = pyvacon.instruments.CouponDescription(refdate + dt.timedelta(days = 365), refdate + dt.timedelta(days = 2*365),
                                     refdate + dt.timedelta(days = 2*365), enums.DayCounter.ACT365_FIXED, 0.05)

coupons = pyvacon.analytics.vectorCouponDescription()
coupons.append(coupon)
inflationLag = 0
deflationProtection = False
pricing_data_infl.spec = pyvacon.instruments.InflationLinkedBondSpecification('TEST_INFL', 'BMW', enums.SecuritizationLevel.SENIOR_SECURED, 'EUR', 
                                           refdate + dt.timedelta(days = 20), refdate + dt.timedelta(days = 2*365), 100.0, 
                                            [coupon], inflationLag, enums.InflationInterpolation.GERMAN,
                                            'CPI', deflationProtection)

#### Pricecall

In [ ]:
results = pyvacon.pricing.price(pricing_data_infl)
print(pricing_data_infl.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()))

#### Outputs
##### Macaulay Duration
Macaulay duration, named for Frederick Macaulay who introduced the concept, is the weighted average maturity of cash flows. Consider some set of fixed cash flows. The present value of these cash flows is:

$$ V=\sum _{i=1}^{n}PV_{i}V=\sum _{i=1}^{n}PV_{i}$$
The Macaulay duration is defined as:
$$ MacD={\frac {\sum _{i=1}^{n}{t_{i}PV_{i}}}{\sum _{i=1}^{n}{PV_{i}}}}={\frac {\sum _{i=1}^{n}{t_{i}PV_{i}}}{V}}=\sum _{i=1}^{n}t_{i}{\frac {PV_{i}}{V}}$$
where:

- $i$ indexes the cash flows,
- $PV_{i}$ is the present value of the $i$th cash payment from an asset,
- $t_{i}$ is the time in years until the $i$th payment will be received,
- $V$ is the present value of all future cash payments from the asset.

**NOTE:** To compute the year fraction $t_i$ we use the daycounter of the underlying discount curve.

**TODO**:Explain Macaulay duration for JLT.
##### Modified Duration and Rho
In contrast to Macaulay duration, modified duration (sometimes abbreviated MD) is a price sensitivity measure, defined as the percentage derivative of price with respect to yield (the logarithmic derivative of bond price with respect to yield). Modified duration applies when a bond or other asset is considered as a function of yield. In this case one can measure the logarithmic derivative with respect to yield:

$$ ModD(y)\equiv -{\frac {1}{V}}\cdot {\frac {\partial V}{\partial y}}=-{\frac {\partial \ln(V)}{\partial y}}$$

When the yield is expressed continuously compounded, Macaulay duration and modified duration are numerically equal. To see this, if we take the derivative of price or present value, expression (2), with respect to the continuously compounded yield $y$ we see that:

$$\frac {\partial V}{\partial y}=-\sum _{i=1}^{n}t_{i}\cdot CF_{i}\cdot e^{-y\cdot t_{i}}=-MacD\cdot V,$$
In other words, for yields expressed continuously compounded,
$ModD=MacD$.

We apply the above equation with the difference that instead of the flat yield we use the time dependent zero rates $r(t_i)$ from the discount curve used to discount the cash flows:
$$\frac {\partial V}{\partial y}=-\sum _{i=1}^{n}t_{i}\cdot CF_{i}\cdot e^{-r(t_i)\cdot t_{i}}=-MacD\cdot V,$$.
To be consistent with the equity part we store this modified duration as rho in the PricingResults object.

In [ ]:
pricing_data_infl.pricingRequest.setMacaulayDuration(True)
pricing_data_infl.pricingRequest.setRho(True)
results = pyvacon.pricing.price(pricing_data_infl)
print(pricing_data_infl.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  Macaulay Duration: " + str(results.getMacaulayDuration()) + ",  Mod Duration/rho: " + str(results.getRho('EONIA')))

##### Convexity
Convexity is the second derivative of the price w.r.t. the rates, measuring.

In [ ]:
pricing_data_infl.pricingRequest.setConvexity(True)
results = pyvacon.pricing.price(pricing_data_infl)
print(pricing_data_infl.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice())
      + ", convexity: " + str(results.getConvexity()))

##### Yield to maturity (YTM)
The yield to maturity (YTM), book yield or redemption yield of a bond is the (theoretical) internal rate of return (IRR, overall interest rate) earned by an investor who buys the bond today at the market price, assuming that the bond is held until maturity, and that all coupon and principal payments are made on schedule.

The YTM is often given in terms of Annual Percentage Rate (A.P.R.), but more often market convention is followed. In a number of major markets (such as gilts) the convention is to quote annualized yields with semi-annual compounding (see compound interest); thus, for example, an annual effective yield of 10.25% would be quoted as 10.00%, because 1.05 × 1.05 = 1.1025 and 2 × 5 = 10.

**Note that the YTM computed by the BondPricer is the continuously compounded rate w.r.t. the daycounter of the respective discount curve used for pricing the product.**

In [ ]:
pricing_data_infl.pricingRequest.setYTM(True)
results = pyvacon.pricing.price(pricing_data_infl)
print(pricing_data_infl.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  ytm: " + str(results.getYTM()))

##### Theta
Theta describes the change of value 

In [ ]:
pricing_data_infl.pricingRequest.setYTM(False) #switch of ytm calculation
pricing_data_infl.pricingRequest.setTheta(True)
results = pyvacon.pricing.price(pricing_data_infl)
print(pricing_data_infl.spec.getObjectId() + ', dirty price: ' + str(results.getPrice()) + ",  clean price: " + str(results.getCleanPrice()), 
     ",  theta: " + str(results.getTheta()))